In [1]:
# %matplotlib inline
%matplotlib
import re

import numpy as np
import pandas as pd
import seaborn as sns

import ggplot as gp
import matplotlib.pyplot as plt

import munch

import vcf

Using matplotlib backend: TkAgg


In [2]:
# set figure characteristics

# size
sns.set_context("poster")

# Grid style
sns.set_style("whitegrid")

# colors
sns.set_palette(sns.hls_palette(n_colors=5, h=0.59, l=0.4, s=0.75))

ggplot_theme = gp.theme_seaborn(context='poster')
ggplot_theme._rcParams['axes.labelsize'] = 26
ggplot_theme._rcParams['font.size'] = 20

color_list = ["#0000FF","#00FF00","#000000"]

site_cmap =     {"KG": "#FF0000",
                 "OT": "#000000",
                 "MS": "#0000FF",
                 "NB": "#00FF00"}
manual_color_scale = gp.scale_color_manual(values=color_list)



In [3]:
# ggplot_theme._rcParams

In [4]:
# Paths
vcf_path = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/tsetseFINAL_14Oct2014_f2_53.recode.renamed_scaffolds.maf0_05.OT_MS_NB_indv.recode.vcf"

xls_with_infection_status = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-07-28_infection_status/Infection_Glossina_ddRADs_STATUS.xls"

pcadapt_err = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/PCAdapt/results/PCAdapt_s1000_b500_k{k}.stats"

pcadapt_dir = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/PCAdapt/results"

pcadapt_snps = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/tsetseFINAL_14Oct2014_f2_53.recode.renamed_scaffolds.maf0_05.OT_MS_NB_indv.recode.vcfsnp"



# # Paths
# vcf_path = "/run/media/gus/Storage/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/tsetseFINAL_14Oct2014_f2_53.recode.renamed_scaffolds.maf0_05.OT_MS_NB_indv.recode.vcf"

# xls_with_infection_status = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-07-28_infection_status/Infection_Glossina_ddRADs_STATUS.xls"

# pcadapt_err = "/run/media/gus/Storage/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/PCAdapt/results/PCAdapt_s1000_b500_k{k}.stats"

# pcadapt_dir = "/run/media/gus/Storage/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/PCAdapt/results"

# pcadapt_snps = "/run/media/gus/Storage/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/tsetseFINAL_14Oct2014_f2_53.recode.renamed_scaffolds.maf0_05.OT_MS_NB_indv.recode.vcfsnp"

In [5]:
def get_errors():
    errors = munch.Munch({'k':[], 'error':[]})
    
    for i in range(10):
        k = i+1
        with open(pcadapt_err.format(k=k), 'rU') as stats:
            errors.k.append(k)
            errors.error.append(float(stats.readline().rstrip('\n').split(' ')[-1]))
    return pd.DataFrame(errors)

def recode_column(column_name):
    
    if isinstance(column_name,int):
        i = column_name + 1
        return "Factor {i}".format(i=i)
    
    if column_name.upper() == "INDEX":
        return "sample"
    
def load_scores(pcadapt_scores_path, vcf_path):
    vcf_reader = vcf.Reader(open(vcf_path, 'r'))
    indiv_names = vcf_reader.samples
    
    df = pd.read_table(filepath_or_buffer=pcadapt_scores_path, sep='\s', 
                                             header=None, names=indiv_names,
                                             engine='python').T.reset_index()
    df = df.rename(columns=recode_column)
    
    return df

def load_main(pcadapt_path):
    
    return pd.read_table(filepath_or_buffer=pcadapt_path, sep='\t', engine='python').dropna(axis=1)

def recode_infection_data(df):
    
    sample_func = lambda name: "{collection}_{individual}".format(collection=name.replace('_','')[:4],
                                                                  individual=name.replace('_','')[4:])
    infection_func = lambda status: "*" if status == 'yes' else ""
    
    df["Sample"] = df.Sample.apply(sample_func)
    df["Infection"] = df.Infection.apply(infection_func)
    
    return df

In [6]:
# # load errors
# errors = get_errors()
# errors

IOError: [Errno 2] No such file or directory: '/run/media/gus/Storage/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/PCAdapt/results/PCAdapt_s1000_b500_k1.stats'

In [ ]:
# gp_errors = gp.ggplot(errors, gp.aes('k', 'error')) \
#     + gp.geom_point(size=80) \
#     + ggplot_theme

# my_plots["gp_errors"] = gp_errors 
# gp_errors

In [8]:
# load infection status tables
infection_data = pd.read_excel(xls_with_infection_status, sheetname="InfectionState")
infection_data = recode_infection_data(infection_data[["Sample","Infection"]].dropna())
infection_data.head(10)

,Sample,Infection
0,KG10_044,*
1,KG10_057,*
2,OT11_0052,*
3,OT11_0072,*
4,OT11_0073,*
5,OT11_0079,*
6,OT11_0047,
7,OT11_0048,
8,OT11_0049,
9,NB11_005,*


In [9]:
# load scores
PCAdapt_s1000_b500_k2_scores = load_scores(pcadapt_scores_path=pcadapt_dir+"/PCAdapt_s1000_b500_k2.scores",
                                           vcf_path=vcf_path)


PCAdapt_s1000_b500_k2_scores.head()

IOError: [Errno 2] No such file or directory: '/run/media/gus/Storage/louise/data/genomes/glossina_fuscipes/annotations/SNPs/vcftools_out/ddrad58_populations/individuals/tsetseFINAL_14Oct2014_f2_53.recode.renamed_scaffolds.maf0_05.OT_MS_NB_indv.recode.vcf'

In [ ]:
# Load main pcadapt result file
PCAdapt_s1000_b500_k2 = load_main(pcadapt_path=pcadapt_dir+"/PCAdapt_s1000_b500_k2")
PCAdapt_s1000_b500_k2.head()

# Plotting functions

In [ ]:
# set labels
def set_site_names(df):
    re_site = re.compile('\A\D{,3}')
    df["site"] = df['sample'].apply(lambda indiv: re_site.findall(indiv)[0])
    df.sort(columns='sample', inplace=True)
    df.reset_index(inplace=True)
    df["individuals"] = df.index.values + 1

    return df.drop(["index"], axis=1)

# plot factors vs individuals

def plot_factors_v_indivs(df):
    
    plt.figure(figsize=(20, 6))
    f = sns.stripplot(x="Factor 2", y="Factor 1", hue="site", data=df, jitter=False, split=True)
#     f = plt.scatter(x=df["Factor 1"], y=df["Factor 2"], c=df["site"],cmap=sns.cmap)
    return f
    


In [ ]:
my_plots = {}

In [ ]:
PCAdapt_s1000_b500_k2_scores = set_site_names(PCAdapt_s1000_b500_k2_scores)

In [ ]:
PCAdapt_s1000_b500_k2_scores.sort("Factor 2", inplace=True)
PCAdapt_s1000_b500_k2_scores.head(15)

In [ ]:
# join infection state to the scores table
PCAdapt_s1000_b500_k2_scores = PCAdapt_s1000_b500_k2_scores.merge(right= infection_data, 
                                                                  how='left', 
                                                                  on=None, left_on="sample", right_on="Sample", 
                                                                  left_index=False, right_index=False, 
                                                                  sort=False, suffixes=('_x', '_y'), 
                                                                  copy=True)
PCAdapt_s1000_b500_k2_scores.head()

In [ ]:
gp_factor1 = gp.ggplot(PCAdapt_s1000_b500_k2_scores, gp.aes('individuals', 'Factor 1', color='site')) \
    + gp.geom_point(size=80) \
    + ggplot_theme + manual_color_scale

my_plots["gp_factor1"] = gp_factor1 
gp_factor1

In [ ]:
gp_factor2 = gp.ggplot(PCAdapt_s1000_b500_k2_scores, gp.aes('individuals', 'Factor 2', color='site')) \
    + gp.geom_point(size=80) \
    + ggplot_theme + manual_color_scale
my_plots["gp_factor2"] = gp_factor2
gp_factor2

In [ ]:
# plt.figure(figsize=(12,12))
gp_factor1x2 = gp.ggplot(PCAdapt_s1000_b500_k2_scores, gp.aes('Factor 1', 'Factor 2', color='site')) 
gp_factor1x2 += gp.geom_point(size=80)
gp_factor1x2 += ggplot_theme
gp_factor1x2 += manual_color_scale
my_plots["gp_factor1x2"] = gp_factor1x2
gp_factor1x2

In [ ]:
gp_factor1x2_inf = gp.ggplot(PCAdapt_s1000_b500_k2_scores, gp.aes('Factor 1', 'Factor 2', color='site', label="Infection")) 
gp_factor1x2_inf += gp.geom_point(size=80)
gp_factor1x2_inf += ggplot_theme
gp_factor1x2_inf += gp.geom_text(color="m",size=20)
gp_factor1x2_inf += manual_color_scale
my_plots["gp_factor1x2_inf"] = gp_factor1x2_inf
gp_factor1x2_inf

In [ ]:
def get_factors_rename_map(df):    
    column_map = {}
    for k,v in zip(df.columns, list(np.arange(0,len(df.columns))+1)):
        column_map[k] = "Factor {v}".format(v=str(v))

    return column_map

def assign_factors_to_snps(df):
    factor_ps = df.iloc[:,2:]
    factor_name_map = get_factors_rename_map(factor_ps)
    factor_ps = factor_ps.rename(columns=factor_name_map)
    
    factor_id = factor_ps.apply(lambda x: x.argmax(), axis = 1)
    
    df['Factor'] = factor_id
    df.rename(columns=factor_name_map, inplace=True)
    
    return df

def add_snp_number(df):
    df["SNP"] = df.index +1
    return df

def plot_factor_SNPs(df):
    pass
    

In [ ]:
PCAdapt_s1000_b500_k2.head()

In [ ]:
PCAdapt_s1000_b500_k2_ = PCAdapt_s1000_b500_k2.copy()
PCAdapt_s1000_b500_k2_.head()

In [ ]:
# pdb

In [ ]:
f = assign_factors_to_snps(PCAdapt_s1000_b500_k2_)

In [ ]:
f.head(18)

In [ ]:
f = add_snp_number(f)

In [ ]:
f.head()

In [ ]:
zones = pd.DataFrame(data={'Strength of evidence':["Worth a mention","Substantial"],
                           'logBF':[0,0.5],
                           'SNP':[75000,75000],
                          })

In [ ]:
zones.head()

In [ ]:
aes_lines = gp.aes(#linetype=None,
                   color="Strength of evidence",
                   x='SNP',
                   y='logBF', 
                   ) 

aes_dots = gp.aes(x='SNP', 
                  y='logBF', 
                  color='Factor')

In [ ]:
lines = gp.geom_line(aes_lines, 
                     size=3, 
                     data=zones)

dots = gp.geom_point(aes_dots, 
                     alpha=0.6, 
                     data=f)

In [ ]:
# gp_local_adapt = gp.ggplot(aes_dots,f)
# gp_local_adapt += gp.theme_seaborn()
# my_plots["gp_local_adapt"] = gp_local_adapt
# gp_local_adapt + lines + dots + ggplot_theme

In [ ]:
gp_local_adapt = gp.ggplot(f, gp.aes('SNP', 'logBF', color='Factor', xmin=0, xmax=len(f)))
gp_local_adapt += gp.geom_point(alpha=0.6, size=60)
gp_local_adapt += gp.geom_hline(yintercept=[0], show_guide=True, color='k', linetype='--', size=3)
# gp_local_adapt += gp.geom_text(gp.aes(x="x", y="y", label="LineKind"),data=zones)
gp_local_adapt += ggplot_theme
my_plots["gp_local_adapt"] = gp_local_adapt

gp_local_adapt

# Identify which SNPs are the probable outliers

In [33]:
snp_outliers = f.query("logBF >= 0").sort('logBF',ascending=False)
snp_outliers

,logBF,logPO,Factor 1,Factor 2,Factor,SNP
22488,0.145904,-2.809855,0.022371,0.977629,Factor 2,22489
22487,0.145718,-2.810041,0.022444,0.977556,Factor 2,22488
40971,0.098694,-2.857064,0.025200,0.974800,Factor 2,40972
22278,0.054760,-2.900998,0.028083,0.971917,Factor 2,22279
39523,0.012381,-2.943377,0.030552,0.969448,Factor 2,39524
55935,0.001190,-2.954569,0.031516,0.968484,Factor 2,55936


In [34]:
snps = pd.read_table(pcadapt_snps, sep=' ', header=None)

In [35]:
snps["SNP"] = snps.index.values + 1

In [36]:
snps.head()

,0,1,2,3,4,5,6,7,8,9,SNP
0,Scaffold0,13388,.,T,C,999,.,.,GT:PL:DP:SP:GQ,NaN,1
1,Scaffold0,23408,.,T,G,999,.,.,GT:PL:DP:SP:GQ,NaN,2
2,Scaffold0,23418,.,G,C,999,.,.,GT:PL:DP:SP:GQ,NaN,3
3,Scaffold0,23421,.,T,C,999,.,.,GT:PL:DP:SP:GQ,NaN,4
4,Scaffold0,23431,.,G,A,999,.,.,GT:PL:DP:SP:GQ,NaN,5


In [44]:
snps_joined_location = f.join(snps[range(0,10)], how='inner', lsuffix='outlier')
snps_joined_location.head()

,logBF,logPO,Factor 1,Factor 2,Factor,SNP,0,1,2,3,4,5,6,7,8,9
0,-0.879953,-3.835711,0.809018,0.190982,Factor 1,1,Scaffold0,13388,.,T,C,999,.,.,GT:PL:DP:SP:GQ,NaN
1,-0.951201,-3.906960,0.786001,0.213999,Factor 1,2,Scaffold0,23408,.,T,G,999,.,.,GT:PL:DP:SP:GQ,NaN
2,-0.913359,-3.869118,0.800658,0.199342,Factor 1,3,Scaffold0,23418,.,G,C,999,.,.,GT:PL:DP:SP:GQ,NaN
3,-0.913434,-3.869192,0.800617,0.199383,Factor 1,4,Scaffold0,23421,.,T,C,999,.,.,GT:PL:DP:SP:GQ,NaN
4,-0.921170,-3.876929,0.798722,0.201278,Factor 1,5,Scaffold0,23431,.,G,A,999,.,.,GT:PL:DP:SP:GQ,NaN


In [49]:
snps_joined_location.to_csv(pcadapt_dir+'/snps_joined_location.csv', index=False)

In [37]:
snp_outliers_joined_location = snp_outliers.join(snps, on='SNP', how='inner', lsuffix='outlier')
snp_outliers_joined_location

,logBF,logPO,Factor 1,Factor 2,Factor,SNPoutlier,0,1,2,3,4,5,6,7,8,9,SNP
22488,0.145904,-2.809855,0.022371,0.977629,Factor 2,22489,Scaffold150,13773,.,G,A,999,.,.,GT:PL:DP:SP:GQ,NaN,22490
22487,0.145718,-2.810041,0.022444,0.977556,Factor 2,22488,Scaffold150,13772,.,G,A,999,.,.,GT:PL:DP:SP:GQ,NaN,22489
40971,0.098694,-2.857064,0.025200,0.974800,Factor 2,40972,Scaffold250,396214,.,G,C,999,.,.,GT:PL:DP:SP:GQ,NaN,40973
22278,0.054760,-2.900998,0.028083,0.971917,Factor 2,22279,Scaffold144,565217,.,A,T,999,.,.,GT:PL:DP:SP:GQ,NaN,22280
39523,0.012381,-2.943377,0.030552,0.969448,Factor 2,39524,Scaffold207,475992,.,A,C,999,.,.,GT:PL:DP:SP:GQ,NaN,39525
55935,0.001190,-2.954569,0.031516,0.968484,Factor 2,55936,Scaffold368,310507,.,T,C,999,.,.,GT:PL:DP:SP:GQ,NaN,55937


In [42]:
snp_outliers_joined_location.to_clipboard()

In [38]:
snp_outliers_joined_location[["logBF",0,1]]

,logBF,0,1
22488,0.145904,Scaffold150,13773
22487,0.145718,Scaffold150,13772
40971,0.098694,Scaffold250,396214
22278,0.054760,Scaffold144,565217
39523,0.012381,Scaffold207,475992
55935,0.001190,Scaffold368,310507


In [39]:
# create bed string 

snp_outliers_joined_location_bed = snp_outliers_joined_location[[0,1]]
snp_outliers_joined_location_bed.columns = ["Seq","end"]
snp_outliers_joined_location_bed["start"] = snp_outliers_joined_location_bed['end'] - 1
snp_outliers_joined_location_bed

-c:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Seq,end,start
22488,Scaffold150,13773,13772
22487,Scaffold150,13772,13771
40971,Scaffold250,396214,396213
22278,Scaffold144,565217,565216
39523,Scaffold207,475992,475991
55935,Scaffold368,310507,310506


In [40]:
print snp_outliers_joined_location_bed.to_string()

               Seq     end   start
22488  Scaffold150   13773   13772
22487  Scaffold150   13772   13771
40971  Scaffold250  396214  396213
22278  Scaffold144  565217  565216
39523  Scaffold207  475992  475991
55935  Scaffold368  310507  310506
